# Online store A/B test
We've got some hypotheses meant to boost revenue that need to be prioritised, then we'll do an A/B test and obviously that'll be analysed. That's it.

We'll have three tables to work with:
* hypotheses_us.csv, the hypotheses along with some scores for calculation of RICE and ICE
* orders_us.csv, the orders during the A/B test, with who made them on what day, which group they were part of, and how much they spent
* visits_us.csv, how many visits were assigned to each group on each day of the test

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from scipy import stats as st
import seaborn as sns

hypo = pd.read_csv("datasets/hypotheses_us.csv", sep=";")
orders = pd.read_csv("datasets/orders_us.csv", dtype={"group": "category"}, parse_dates=["date"])
visits = pd.read_csv("datasets/visits_us.csv", dtype={"group": "category"}, parse_dates=["date"])

## Part 0: Looking at the tables and checking for weird stuff

In [ ]:
hypo

Title case column names, how annoying. Other than that, looks fine. Also, I want to see what these proposals actually are.

In [ ]:
pd.set_option("display.max_colwidth", None)
hypo["Hypothesis"]

In [ ]:
orders.info()
orders.head(12)

"Id". That's annoying too.

Revenue has one decimal place. Not zero and not two. Not really a problem, but kind of weird. I guess everything was just a multiple of ten cents. Or pence, or kopeyki.

What is a problem is that this is about the whole of August 2019 and we'll be doing some cumulative stats, and here we are starting in the middle, so let's fix that.

In [ ]:
orders = orders.sort_values("date")

In [ ]:
visits.info()
visits.head(12)

This is fine.

Now let's rename these columns or I'll definitely type them wrong a lot.

In [ ]:
hypo = hypo.rename(str.lower, axis=1)
orders = orders.rename(columns={"transactionId": "transaction_id", "visitorId": "visitor_id"})

In [ ]:
print(orders["transaction_id"].nunique())

In [ ]:
print(orders[orders["revenue"] <= 0])

In [ ]:
test = orders.groupby("visitor_id").agg({"group": "nunique"})
print(test[test["group"] != 1])

That seems like a lot. How many visitors total do we have?

In [ ]:
print(orders["visitor_id"].nunique())

So that's...

In [ ]:
print((test[test["group"] != 1]["group"].count()) / 1031)

Over 5% of visitors being in both groups. We'll... just ignore those guys. At least ignore their orders. It'd be nice if the visits table was actually a table with each visit.

In [ ]:
vgnums = orders.groupby("visitor_id", as_index=False).agg({"group": "nunique"})
dualgrouped = vgnums[vgnums["group"] != 1]["visitor_id"]
orders = orders[~orders["visitor_id"].isin(dualgrouped)]

## Part 1: Ranking the hypotheses
For this we'll use methods called ICE and RICE. RICE stands for Reach, Impact, Confidence, Effort and ICE is simply RICE without reach.

ICE is (Impact * Confidence) / Effort. That is, we rate an idea for the impact it would have if it worked well, how likely it is to work well, and how much time, resource, and cleverness it would take to implement. For example, a hypothetical feature would almost definitely have a decent impact if we added it, but actually building it would be rather impractical, so its good impact and great confidence would be divided by the high effort score and leave it with a middling overall score. RICE simply adds reach to the multiplication, representing what fraction of the users it would be expected to affect.

In [ ]:
hypo["ice"] = hypo["impact"] * hypo["confidence"] / hypo["effort"]
hypo["rice"] = hypo["reach"] * hypo["impact"] * hypo["confidence"] / hypo["effort"]
coldsort = hypo.sort_values(by="ice", ascending=False)
tastysort = hypo.sort_values(by="rice", ascending=False)
display(coldsort)
display(tastysort)

The birthday promotion wins on ICE, but the fact that it'll only exist for a certain user 1/365th of the time (or a quarter as often for the Feb 29ers out there) knocks it down in RICE. The big winners in RICE compared to ICE are the recommendation blocks, which jumps three places because of course most people are going to see that, and making the subscription form be more prominent or perhaps exist at all, which gets first place because everyone will see that, though it was so good anyway that even a middling reach would've brought it victory. We could speculate about how many people will actually use these things when they see them, but the scoring isn't part of our job.

We could also do it visually. Here's a scatterplot of the ICEs versus the RICEs. Note the practically reachless birthday idea way in the upper left corner.

In [ ]:
fig = px.scatter(hypo, x="rice", y="ice", hover_data=["hypothesis", "ice", "rice"])
fig.update_layout(xaxis_title="RICE", yaxis_title="ICE", title="Subscription form wins by a long way")
fig.show()

Have a pair plot too.

In [ ]:
sns.pairplot(hypo, x_vars=["reach", "impact", "confidence", "effort"], y_vars=["reach", "impact", "confidence", "effort"], corner=True)

In the effort row, that lower right corner is where you want to be, a high effect (or confidence of having one) with low effort. That sounds like it'd be a tradeoff, and of course it is, but some simple but effective (at least effective-sounding) ideas manage to do well. Also, note the lack of ideas that are both high-impact and either low-confidence or high-effort, no "big risk, big reward" things. Which is fine, we wouldn't be too interested in them anyway.

## Part 2: A/B test analysis
We did the A/B test! You didn't get to see it and neither did I, but as you can see from the existence of the group columns in the other tables, it happened. So I kind of did things in backwards time. Anyway, let's analyse it.

First of all, cumulative revenue.

In [ ]:
day_revenues = orders.groupby(["date", "group"], as_index=False).agg({"revenue": "sum"})

day_revenues["revenue_cum"] = day_revenues.groupby("group").cumsum()
fig = px.line(day_revenues, x="date", y="revenue_cum", color="group")
fig.update_layout(xaxis_title="Date", yaxis_title="Cumulative revenue", legend_title_text="Group", title='Group B "wins" on an outlier')
fig.show()

Group B had one really good day, but other than that it was quite even. Though even not counting the big day, group B was slightly better.

Next up, average order size.

In [ ]:
orders["cumsum"] = orders.groupby("group")["revenue"].cumsum()
orders["cumcount"] = orders.groupby("group")["revenue"].cumcount() + 1
orders["cumavg"] = orders["cumsum"] / orders["cumcount"]
day_end_stats = orders.drop_duplicates(subset=["date", "group"], keep="last")
del day_end_stats["transaction_id"]
del day_end_stats["visitor_id"]
del day_end_stats["revenue"]

fig = px.line(day_end_stats, x="date", y="cumavg", color="group")
fig.update_layout(xaxis_title="Date", yaxis_title="Cumulative average order size", legend_title_text="Group", title='Group B "wins" on the same outlier')
fig.show()

You think we should have a look at the distributions?

In [ ]:
fig = go.Figure(data=[
    go.Histogram(name="Group A", x=orders[orders["group"] == "A"]["revenue"]),
    go.Histogram(name="Group B", x=orders[orders["group"] == "B"]["revenue"])
])
fig.update_layout(xaxis_title="Revenue", yaxis_title="Number of orders", title="This is why you don't always trust plain means")
fig.show()

Yeah, that's about what I expected. Let's zoom in on the ones less than 1000.

In [ ]:
fig = go.Figure(data=[
    go.Histogram(name="Group A", x=orders[(orders["group"] == "A") & (orders["revenue"] < 1000)]["revenue"]),
    go.Histogram(name="Group B", x=orders[(orders["group"] == "B") & (orders["revenue"] < 1000)]["revenue"])
])
fig.update_layout(xaxis_title="Revenue", yaxis_title="Number of orders", title="This is why you don't always trust plain means")
fig.show()

We could also get some actual percentile cutoffs.

In [ ]:
orders["revenue"].describe(percentiles=[.05, .10, .90, .95, .99])

300 seems like the best decimally round number to cut it at. From here on, orders of 300 or more will count as being 300. The meaningfulness of the test depends on it.

### Let's try this again
#### Basic stats

In [ ]:
orders_orig = orders.copy()
orders["revenue"] = orders["revenue"].where(orders["revenue"] <= 300, 300)

day_revenues = orders.groupby(["date", "group"], as_index=False).agg({"revenue": "sum"})

day_revenues["revenue_cum"] = day_revenues.groupby("group").cumsum()
fig = px.line(day_revenues, x="date", y="revenue_cum", color="group")
fig.update_layout(xaxis_title="Date", yaxis_title="Cumulative revenue", legend_title_text="Group", title="Group B has brought 13% more revenue")
fig.show()

In [ ]:
v_grouped = visits.groupby("group").agg({"visits": "sum"})
v_grouped

Group B has brought in 13% more money with only 1% more visits. Very nice. But as important as simple revenue is, we've got more metrics than that. Here's average order size.

In [ ]:
orders["cumsum"] = orders.groupby("group")["revenue"].cumsum()
orders["cumcount"] = orders.groupby("group")["revenue"].cumcount() + 1
orders["cumavg"] = orders["cumsum"] / orders["cumcount"]
day_end_stats = orders.drop_duplicates(subset=["date", "group"], keep="last")
del day_end_stats["transaction_id"]
del day_end_stats["visitor_id"]
del day_end_stats["revenue"]

fig = px.line(day_end_stats, x="date", y="cumavg", color="group")
fig.update_layout(xaxis_title="Date", yaxis_title="Running average order size", legend_title_text="Group", title="Mean order size: the last lead change was a few days ago")
fig.show()

And in relative terms:

In [ ]:
group_a = day_end_stats[day_end_stats["group"] == "A"]
group_b = day_end_stats[day_end_stats["group"] == "B"]
merge = group_a.merge(group_b, how="left", on="date", suffixes=["_a", "_b"])

fig = go.Figure(data=[
    go.Scatter(mode="lines", name="Difference", x=merge["date"], y=(merge["cumavg_a"] / merge["cumavg_b"] - 1) * 100)
])
fig.update_layout(title="Mean order size: the last lead change was a few days ago", xaxis_title="Date", yaxis_title="Group A % advantage over group B")
fig.show()

Not much to choose here. Going by this, the test should probably continue to run. It's close, and the line isn't *that* flat, having recently seen a swing of 5.8 percentage points in three days. But again, we have more things to look at.

Next "round", conversion rate.

In [ ]:
ov_by_day = orders.groupby(["date", "group"], as_index=False).agg({"transaction_id": "count"})
ov_by_day = ov_by_day.merge(visits, on=["date", "group"], how="inner")
ov_by_day = ov_by_day.rename(columns={"transaction_id": "orders"})
ov_by_day["conversion"] = ov_by_day["orders"] / ov_by_day["visits"]

ov_by_day["orders_cum"] = ov_by_day.groupby("group")["orders"].cumsum()
ov_by_day["visits_cum"] = ov_by_day.groupby("group")["visits"].cumsum()
ov_by_day["conversion_cum"] = ov_by_day["orders_cum"] / ov_by_day["visits_cum"]

fig = px.line(ov_by_day, x="date", y="conversion_cum", color="group")
fig.update_layout(title="Group B clearly has higher conversion", xaxis_title="Date", yaxis_title="Running conversion rate", legend_title_text="Group")
fig.show()

Not much to say; group B is just better. 16% better, to be specific. It did decline a bit here in the last couple weeks, but not nearly by enough to lose the lead.

Next I've been asked to make a scatterplot for orders per user. A bar chart would be the most obvious way to do it, but it doesn't matter:

In [ ]:
visitor_groups = pd.DataFrame({"visitor_id": orders["visitor_id"], "group": orders["group"]})
visitor_groups = visitor_groups.drop_duplicates()

grouped = orders.groupby("visitor_id", as_index=False).agg({"transaction_id": "count"})
grouped = grouped.merge(visitor_groups, how="left", on="visitor_id")

fig = px.scatter(grouped, x="visitor_id", y="transaction_id", color="group")
fig.update_layout(title="1 is the only normal number of visits for a user", xaxis_title="Visitor ID", yaxis_title="Orders", legend_traceorder="reversed", legend_title_text="Group")
fig.show()

The message is clear: anything that isn't 1 is an outlier. That said, there aren't any massive outliers to throw off simple calculations either.

In [ ]:
grouped["transaction_id"].describe(percentiles=[.01, .05, .95, .99])

And now with order size:

In [ ]:
fig = px.scatter(orders, x="transaction_id", y="revenue", color="group")
fig.update_layout(title="Orders get noticeably thin from 60 up", xaxis_title="Transaction ID", yaxis_title="Revenue", legend_title_text="Group")
fig.show()

Of course, not too many of those 300 orders were actually 300, but doing it with the original has the presence of that one 20k order squishing the rest down and making the chart pretty useless. Also, I *really* feel like there should be a bar chart of this, so here it is.

In [ ]:
fig = px.histogram(orders, x="revenue", nbins=38)
fig.update_layout(title="The oddly straight slope down to the 70s", xaxis_title="Revenue", yaxis_title="Number of orders")
fig.show()

#### Statistical significance tests
Now we run some actual mathematical tests to see if the differences are that big, as opposed to me looking at pictures and making fleshy judgments.

In [ ]:
import numpy as np #just for this one thing near the end
# The test we're using is quite robust to outliers (like, it's ranked), so don't need the "normal orders only" versions of the tables to have "normal" quite as narrow
cutoff = np.percentile(orders_orig["revenue"], 95)
orders_small = orders_orig[orders_orig["revenue"] <= cutoff]
# Not fixing the normal orders only cumulative values, because they won't be needed

ovs_by_day = orders_small.groupby(["date", "group"], as_index=False).agg({"transaction_id": "count"})
ovs_by_day = ovs_by_day.merge(visits, on=["date", "group"], how="inner")
ovs_by_day = ovs_by_day.rename(columns={"transaction_id": "orders"})
ovs_by_day["conversion"] = ovs_by_day["orders"] / ovs_by_day["visits"]

ov_by_day_a = ov_by_day[ov_by_day["group"] == "A"]
ov_by_day_b = ov_by_day[ov_by_day["group"] == "B"]
ovs_by_day_a = ovs_by_day[ovs_by_day["group"] == "A"]
ovs_by_day_b = ovs_by_day[ovs_by_day["group"] == "B"]

ovd_a_cv = ov_by_day_a["conversion"]
ovd_b_cv = ov_by_day_b["conversion"]
ovsd_a_cv = ovs_by_day_a["conversion"]
ovsd_b_cv = ovs_by_day_b["conversion"]

First up, the difference in conversion rate, taking all orders into account. To be specific about it, our H<sub>0</sub> is that there's no difference in the conversion rate by day and H<sub>1</sub> is that we do see a difference in conversion rate by day. Specifically what the p-value will be of is that what group B experienced leads to higher conversion.

In [ ]:
alpha = 0.05
results = st.mannwhitneyu(ovd_a_cv, ovd_b_cv, use_continuity=True, alternative="less")
print("p-value:", results.pvalue)
if results.pvalue < alpha:
    print("Statistical significance! (If 5% was a good choice of alpha)")
else:
    print("The null hypothesis remains intact.")

2.7%, nice. Group B's lead in conversion is probably meaningful after all.

Next up is a difference in order size, with all orders, and counting them as their actual values instead of capping them. (Not that capping them or not makes much difference since the Mann-Whitney U test is rank-based.) H<sub>0</sub> here is no effect as going by the raw order sizes – what we're testing against each other is just two lists of how much revenue each order was, one list for each group – and H<sub>1</sub> is that the difference between the groups does have an effect on the order size. The p-value is of group A's advantage, or group B's disadvantage if you want to look at it that way. We don't know which is the control group; this is a properly blinded test, after all.

In [ ]:
results = st.mannwhitneyu(orders_orig[orders_orig["group"] == "A"]["revenue"], orders_orig[orders_orig["group"] == "B"]["revenue"], use_continuity=True, alternative="greater")
print("p-value:", results.pvalue)
if results.pvalue < alpha:
    print("Statistical significance! (If 5% was a good choice of alpha)")
else:
    print("The null hypothesis remains intact.")

No surprise there.

The next test is for a difference in conversion, but leaving out the highest 5% of orders by revenue. Like the other conversion rate test, H<sub>0</sub> is no difference between the conversion rates for each day and H<sub>1</sub> is that there is a significant difference. The p-value again refers to group B having higher conversion.

In [ ]:
alpha = 0.05
results = st.mannwhitneyu(ovsd_a_cv, ovsd_b_cv, use_continuity=True, alternative="less")
print("p-value:", results.pvalue)
if results.pvalue < alpha:
    print("Statistical significance! (If 5% was a good choice of alpha)")
else:
    print("The null hypothesis remains intact.")

Still something to be confident about.

The final Mann-Whitney U test we'll be running is for a difference in order size, but only counting the non-big orders, "non-big" meaning the same as in the previous test. H<sub>0</sub> is no effect based on the order sizes – again, it's just a list of the orders' revenues – and H<sub>1</sub> is that there is a difference between these by group. Like the other order size test, the p-value is for group A having higher order sizes.

In [ ]:
results = st.mannwhitneyu(orders_small[orders_small["group"] == "A"]["revenue"], orders_small[orders_small["group"] == "B"]["revenue"], use_continuity=True, alternative="greater")
print("p-value:", results.pvalue)
if results.pvalue < alpha:
    print("Statistical significance! (If 5% was a good choice of alpha)")
else:
    print("The null hypothesis remains intact.")

The *p*-value is *better*, but being better than 43% isn't anything to write home about.

## Decision time

Whenever there's a difference that inspires much confidence in itself, it's group B in front, and the differences in those revenue and conversion rate lines are looking pretty stable. I'd be happy to stop the test and declare group B superior.